In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import os
from pandas import json_normalize

- read_csv 로 df 변수 안에 데이터 프레임 저장

In [2]:
# 출력 옵션 변경
pd.set_option('display.max_colwidth', None)

# csv 파일 읽고 변수에 저장
df = pd.read_csv('C:/Users/user/Desktop/GA4_project/train.csv')

# 데이터 프레임 확인
df.head(1)

C:\Users\user\AppData\Local\Temp\ipykernel_18764\1612969822.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('C:/Users/user/Desktop/GA4_project/train.csv')


,channelGrouping,date,device,fullVisitorId,geoNetwork,sessionId,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,20160902,"{""browser"": ""Chrome"", ""browserVersion"": ""not available in demo dataset"", ""browserSize"": ""not available in demo dataset"", ""operatingSystem"": ""Windows"", ""operatingSystemVersion"": ""not available in demo dataset"", ""isMobile"": false, ""mobileDeviceBranding"": ""not available in demo dataset"", ""mobileDeviceModel"": ""not available in demo dataset"", ""mobileInputSelector"": ""not available in demo dataset"", ""mobileDeviceInfo"": ""not available in demo dataset"", ""mobileDeviceMarketingName"": ""not available in demo dataset"", ""flashVersion"": ""not available in demo dataset"", ""language"": ""not available in demo dataset"", ""screenColors"": ""not available in demo dataset"", ""screenResolution"": ""not available in demo dataset"", ""deviceCategory"": ""desktop""}",1131660440785968503,"{""continent"": ""Asia"", ""subContinent"": ""Western Asia"", ""country"": ""Turkey"", ""region"": ""Izmir"", ""metro"": ""(not set)"", ""city"": ""Izmir"", ""cityId"": ""not available in demo dataset"", ""networkDomain"": ""ttnet.com.tr"", ""latitude"": ""not available in demo dataset"", ""longitude"": ""not available in demo dataset"", ""networkLocation"": ""not available in demo dataset""}",1131660440785968503_1472830385,Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"", ""bounces"": ""1"", ""newVisits"": ""1""}","{""campaign"": ""(not set)"", ""source"": ""google"", ""medium"": ""organic"", ""keyword"": ""(not provided)"", ""adwordsClickInfo"": {""criteriaParameters"": ""not available in demo dataset""}}",1472830385,1,1472830385


- 데이터 프레임을 넣으면 json 형식을 가지는 컬럼만 새로운 컬럼들로 분해해주는 함수 생성

In [3]:
def convert_all_json_columns(df):
    for column_name, column_data in df.items():
        try:
            # JSON 형식의 문자열을 딕셔너리로 변환하고 분리된 컬럼 추가
            json_columns = pd.json_normalize(column_data.apply(json.loads))
            
            # 기존 데이터프레임에 분리된 컬럼들 추가
            df = pd.concat([df, json_columns], axis=1)
            
            # 기존의 JSON 형식 컬럼 제거
            df = df.drop(columns=[column_name])
        except (json.JSONDecodeError, TypeError):
            # JSON 형식이 아닌 경우 또는 숫자, 문자열 등 예외 처리
            pass
    return df

# 예시 DataFrame 생성
data = {'json_column': ['{"key1": "value1", "key2": "value2"}', '{"key1": "value3", "key2": "value4"}'],
        'other_column': [10, 20],
        'numeric_column': [30, 40]}
df1 = pd.DataFrame(data)

# 함수 적용
df1 = convert_all_json_columns(df1)

# 결과 출력
print(df1)

   other_column  numeric_column    key1    key2
0            10              30  value1  value2
1            20              40  value3  value4


- 실제 데이터 프레임을 집어넣어 실행하고 결과를 json_to_column_df 변수 안에 저장

In [5]:
# 실제 데이터 프레임 삽입
json_to_column_df = convert_all_json_columns(df)

# 결과 확인
json_to_column_df.head(3)

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,browser,browserVersion,...,adwordsClickInfo.criteriaParameters,isTrueDirect,referralPath,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.gclId,adwordsClickInfo.adNetworkType,adwordsClickInfo.isVideoAd,adContent,campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# 남길 컬럼을 확인하기 위해서 키 값 다시 조회
def check_keys(df, column_name):
    all_keys = set()
    for json_str in df[column_name]:
        json_data = json.loads(json_str)
        all_keys.update(json_data.keys())
    
    keys_str = ', '.join(sorted(list(all_keys)))
    result = f"'{column_name}' 컬럼에 들어 있는 키들은 {keys_str} 입니다."
    return result

# 대상 컬럼 : 'device' , 'geoNetwork', 'totals', 'trafficSource'

print(check_keys(df, 'device'))

print(check_keys(df, 'geoNetwork'))

print(check_keys(df, 'totals'))

print(check_keys(df, 'trafficSource'))

'device' 컬럼에 들어 있는 키들은 browser, browserSize, browserVersion, deviceCategory, flashVersion, isMobile, language, mobileDeviceBranding, mobileDeviceInfo, mobileDeviceMarketingName, mobileDeviceModel, mobileInputSelector, operatingSystem, operatingSystemVersion, screenColors, screenResolution 입니다.
'geoNetwork' 컬럼에 들어 있는 키들은 city, cityId, continent, country, latitude, longitude, metro, networkDomain, networkLocation, region, subContinent 입니다.
'totals' 컬럼에 들어 있는 키들은 bounces, hits, newVisits, pageviews, transactionRevenue, visits 입니다.
'trafficSource' 컬럼에 들어 있는 키들은 adContent, adwordsClickInfo, campaign, campaignCode, isTrueDirect, keyword, medium, referralPath, source 입니다.


- 위에서 만든 테이블에서 원하는 컬럼만을 남기고 모두 드랍하는 함수 생성

= 남길 컬럼의 목록 :

1. fullVisitorId
2. visitStarttime
3. channelGrouping
4. continent
5. subContinent
6. country
7. visits
8. pageviews
9. hits
10. newVisits
11. bounces
12. transactionRevenue
13. campaign
14. source
15. medium
16. keyword
17. browser
18. operatingSystem
19. deviceCategory

In [9]:
# 원하는 컬럼만을 가지고 새로운 데이터 프레임을 만드는 함수 생성
def keep_selected_columns(df):
    selected_columns = [
        'fullVisitorId', 'visitStartTime', 'channelGrouping', 'continent', 'subContinent',
        'country', 'visits', 'pageviews', 'hits', 'newVisits', 'bounces',
        'transactionRevenue', 'campaign', 'source', 'medium', 'keyword',
        'browser', 'operatingSystem', 'deviceCategory'
    ]
    
    # 남길 컬럼만을 선택하여 새로운 데이터프레임 생성
    df_selected = df[selected_columns].copy()
    
    return df_selected

In [16]:
# 앞에서 만든 json_to_column_df 를 이용해 새로운 데이터 프레임을 생성하고
# selected_columns_df 변수에 저장
selected_columns_df = keep_selected_columns(json_to_column_df)

# 작동했는지 확인
selected_columns_df.head(3)

,fullVisitorId,visitStartTime,channelGrouping,continent,subContinent,country,visits,pageviews,hits,newVisits,bounces,transactionRevenue,campaign,source,medium,keyword,browser,operatingSystem,deviceCategory
0,1131660440785968503,1472830385,Organic Search,Asia,Western Asia,Turkey,1,1,1,1,1,NaN,(not set),google,organic,(not provided),Chrome,Windows,desktop
1,377306020877927890,1472880147,Organic Search,Oceania,Australasia,Australia,1,1,1,1,1,NaN,(not set),google,organic,(not provided),Firefox,Macintosh,desktop
2,3895546263509774583,1472865386,Organic Search,Europe,Southern Europe,Spain,1,1,1,1,1,NaN,(not set),google,organic,(not provided),Chrome,Windows,desktop


In [18]:
# 테이블 확인 2
selected_columns_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   fullVisitorId       903653 non-null  object
 1   visitStartTime      903653 non-null  int64 
 2   channelGrouping     903653 non-null  object
 3   continent           903653 non-null  object
 4   subContinent        903653 non-null  object
 5   country             903653 non-null  object
 6   visits              903653 non-null  object
 7   pageviews           903553 non-null  object
 8   hits                903653 non-null  object
 9   newVisits           703060 non-null  object
 10  bounces             450630 non-null  object
 11  transactionRevenue  11515 non-null   object
 12  campaign            903653 non-null  object
 13  source              903653 non-null  object
 14  medium              903653 non-null  object
 15  keyword             400724 non-null  object
 16  br

- 결측치가 발생한 newVisits, bounces, transactionRevenue, keyword 의 처리

1. newVisits, bounces, transactionRevenue 는 결측치를 0으로 대치
2. keyword 의 경우에는 NoKeyword 로 결측치를 대치

- 특정 문자열 데이터를 숫자형으로 변경

변경해야할 컬럼
: visits, pageviews, hits, newVisits, bounces, transactionRevenue

- posix 타입으로 되어있는  visitStartTime 컬럼을 UTC 타입으로 변환

: 변환된 데이터를 utc_time 컬럼을 만들어서 저장